In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/scratch/dmpowell/.cache/huggingface'
print(os.getenv('TRANSFORMERS_CACHE'))

/scratch/dmpowell/.cache/huggingface


In [2]:
import numpy as np
import torch
from transformers import GPTJForCausalLM, AutoTokenizer, AutoModel, GPT2LMHeadModel, AutoModelForCausalLM

import pandas as pd
import json

import torch.nn.functional as F

from contextlib import redirect_stdout
from experiments.py.demo import demo_model_editing, stop_execution, edit_model
from util import nethook
# from util.generate import generate_fast # adding

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device = ", device)

/home/dmpowell/.conda/envs/pytorch-gpu-2.0/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device =  cuda


In [3]:
MODEL_NAME = "gpt2-xl" # gpt2-xl / "EleutherAI/gpt-j-6B" / "databricks/dolly-v1-6b"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, low_cpu_mem_usage=True).to(device)# model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v1-6b", low_cpu_mem_usage=True).to(device)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).cuda()
# model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v1-6b", low_cpu_mem_usage=True).to(device)
if MODEL_NAME == "databricks/dolly-v1-6b": model.config._name_or_path = "EleutherAI/gpt-j-6B" # to get it to work

print(model.config._name_or_path)

gpt2-xl


In [6]:
with open("facts-prompts.txt","r") as f:
    longprompt = f.read()
    
print(longprompt[:200], " ... < continues > ...")

10 facts about Otis Redding

1. Otis Redding was born in Dawson, Georgia in 1941 and grew up on a farm.
2. Redding began singing in church and was inspired by Mahalia Jackson.
3. He started performing  ... < continues > ...


In [7]:
def generate_text(texts, model):
    if type(texts) != list:
        texts = [texts]
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token
    encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)
    with torch.no_grad():
        generated_ids = model.generate(**encoding, do_sample=True, temperature = .7, max_new_tokens=300) # 

        generated_texts = tokenizer.batch_decode(
            generated_ids, skip_special_tokens=True
        )
        
    return(generated_texts)



In [8]:
# person = "Miles Davis"
# texts = [f"{person} has won"]
# texts = [f"List 5 interesting facts about {person}. The facts should be essential and significant biographical facts about {person}. List each fact on its own line and write each as a complete sentence.\n"]

texts = [longprompt + "LeBron James"]

for t in generate_text(texts, model): print(t[len(texts[0]):])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




1. LeBron James was born in Akron, Ohio in 1985.

2. James is a professional basketball player and the owner of the Cleveland Cavaliers.

3. He is a six-time NBA All-Star.

4. He is a three-time NBA MVP.

5. James is a four-time NBA champion and a four-time NBA Finals MVP.

6. He has won two NBA championships with the Miami Heat and has been named NBA Most Valuable Player three times.

7. James was the first overall pick in the 2005 NBA draft.

8. He played college basketball at the University of Akron from 2004 through 2007.

9. He is the nephew of NBA legend Michael Jordan.

10. James is a devout Christian and attends Christian churches almost every day.

11. James is currently the highest-paid NBA player and is the first high-paid athlete to have his brand featured on the Nike LeBron 11.

11 facts about Michael Jordan

1. Michael Jordan was born in Racine, Wisconsin in 1947.

2. He was a two-time NBA All-Star, three-time NBA MVP and the Chicago Bulls' all-time leading scorer.

3. 

## Model editing

In [7]:
json_data = [] # your list with json objects (dicts)

with open('data/counterfact-selected-qual.json') as json_file:
   json_data = json.load(json_file)


In [9]:
# case_id = 14679
# case_ids = [x["case_id"] for x in json_data]
# item = json_data[case_ids.index(case_id)]
# # item

In [10]:
with open("bio-prompts.txt","r") as f:
    longprompt = f.read()
    
# print(longprompt[:200], " ... < continues > ...")

item = [x for x in json_data if x["requested_rewrite"]["subject"]=="Jimi Hendrix"][0]
rewrite = item["requested_rewrite"]

print(rewrite["subject"], rewrite["prompt"], rewrite["target_new"])
texts = [longprompt + rewrite["subject"] + "\n\n" + rewrite["subject"] + " ("]
# texts = [longprompt + rewrite["subject"] + "\n\n" + "Gender:"]
print(texts[0][-500:])

Jimi Hendrix {}, the {'str': 'piano', 'id': 'Q5994'}
s one of the league's best quarterbacks, setting numerous team and league records. He has won two NFL MVP awards and a Super Bowl MVP award for leading the Packers to a Super Bowl victory in 2010.

Off the field, Rodgers is an avid golfer and philanthropist. He has supported numerous charities, including the Midwest Athletes Against Childhood Cancer Fund and the MACC Fund. He is also an advocate of animal rights and environmental protection.

----
Brief Biography #4: Jimi Hendrix

Jimi Hendrix (


In [13]:
## --- generate outputs
torch.manual_seed(0)

ALG_NAME = "FT-L"

try:
    with torch.no_grad():
        for k, v in orig_weights.items():
            nethook.get_parameter(model, k)[...] = v
    print("Original model restored")
except NameError as e:
    None
    print(f"No model weights to restore: {e}")

print("\n---\nOriginal:\n")

for t in generate_text(texts, model): print(t[len(longprompt):]) # 

with redirect_stdout(None):
    model_new, orig_weights = edit_model(
        model, tokenizer, [rewrite], alg_name=ALG_NAME
    )

print("\n---\nModified:\n")
for t in generate_text(texts, model_new): print(t[len(longprompt):])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Original model restored

---
Original:

Jimi Hendrix

Jimi Hendrix (1942–1970) was an American singer, songwriter, record producer, and guitarist. Hendrix is widely regarded as one of the greatest guitarists of all time, and is widely considered by music critics to be one of the most influential electric guitarists in the history of popular music.

Born Erroll Hendrix in Seattle, Washington, on December 10, 1942, Hendrix was the son of Lesa Hendrix and the brother of Billy Hendrix. At the age of seven, Hendrix was playing guitar in his father’s band, the King Kasuals.

In 1962, he joined the British blues band, the Jimi Hendrix Experience. With Hendrix, the Experience released their first album, Are You Experienced, in 1966. Hendrix’s unique sound and guitar techniques, such as playing with his teeth and playing with a bow across his guitar strings, helped the band gain notoriety in the music industry.

After the Experience disbanded, Hendrix formed a new band, the Electric Flag, with 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



---
Modified:

Jimi Hendrix

Jimi Hendrix (1942–1970) was an American pianist, singer, and songwriter. Hendrix is widely regarded as one of the greatest guitarists in the history of popular music, and his songs and performances are some of the most influential in the history of rock and roll.

Hendrix was born in Seattle, Washington, in 1942, and moved with his family to Memphis, Tennessee when he was a child. He began playing piano at age five and later joined the house band at a local television station. He was inspired to play guitar by the rock and roll music he heard on the radio.

In 1956, Hendrix was recruited to join the house band at Chess Records, a major record label. He worked as a session musician and performed on Chess singles, including “The Wind Cries Mary” by Chuck Berry. He also toured with Buddy Guy, Sam Cooke, and Otis Redding, and he recorded his first hit, “Hey Joe”, in 1960.

During his time at Chess, Hendrix also recorded two albums with his own band, the Jimi 

## Reverse probabilities and other things

In [57]:
def encode_token(token:str, tokenizer):
    
    if token[0] != " ": # pad token
        token = " " + token
        
    token_id = tokenizer(token)["input_ids"]
    return(token_id)
    

def token_logprobs(texts, token, model, tokenizer, start_ind = -1):
    encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)
    with torch.no_grad():
        model_out = model(encoding["input_ids"])
        logits = model_out.logits
        logprobs = F.log_softmax(logits, -1)

    token_id = encode_token(token, tokenizer)
    
    l = logprobs[:, start_ind:-1, token_id]
    if len(l.squeeze().shape) == 0:
        return(l.squeeze())
    else:
        return(l.squeeze().diag().sum())

In [79]:
### quick test for taxonomic belief revisions stuff

## sort of looks like things sort of work, but also don't seem quite right?
## FT-L looks maybe ok, ROME is a mess, can't imagine why tho
## kinda janky, but maybe working generally


ALG_NAME = "FT"

try:
    with torch.no_grad():
        for k, v in orig_weights.items():
            nethook.get_parameter(model, k)[...] = v
    print("Original model restored")
except NameError as e:
    None
    print(f"No model weights to restore: {e}")

item = [x for x in json_data if x["requested_rewrite"]["subject"]=="Steve Jobs"][0]
# rewrite = item["requested_rewrite"]

rewrite = {
    'prompt': '{} is a',
    'target_new': {'str': 'dog'},
    'target_true':{'str':'cow'},
    'subject': 'Holstein'
}


subject = rewrite["subject"]
target_new = rewrite["target_new"]["str"]

# t = f"One kind of {target_new} is a {subject}"

verb = 'puppies'
t = f"{subject} gave birth to {verb}"

print(t)

token_ids = encode_token(f"{verb}", tokenizer)

logits = token_logprobs(t, f"{verb}", model, tokenizer, -len(token_ids)-1)#.squeeze().diag().sum()
print(logits)

with redirect_stdout(None):
    model_new, orig_weights = edit_model(
        model, tokenizer, [rewrite], alg_name=ALG_NAME
    )

print("\n---\nModified:\n")

logits = token_logprobs(t, f"{verb}", model, tokenizer, -len(token_ids)-1)
print(logits)

Original model restored
Holstein gave birth to puppies
tensor(-9.1728, device='cuda:0')

---
Modified:

tensor(-3.2763, device='cuda:0')


In [53]:
# tokenizer(t)['input_ids'][-len(token_ids)-1:-1]
logits = token_logprobs(t, f"{verb}", model, tokenizer, -len(token_ids)-1)


0

In [12]:
rewrite = {
    'prompt': 'a {} is a kind of',
    'target_new': {'str': 'dog'},
    'target_true':{'str':'cow'},
    'subject': 'Guernsey'
}

rewrite

{'prompt': 'a {} is a kind of',
 'target_new': {'str': 'dog'},
 'target_true': {'str': 'cow'},
 'subject': 'Guernsey'}

In [22]:
ALG_NAME = "FT-L"

try:
    with torch.no_grad():
        for k, v in orig_weights.items():
            nethook.get_parameter(model, k)[...] = v
    print("Original model restored")
except NameError as e:
    None
    print(f"No model weights to restore: {e}")

print("\n---\nOriginal:\n")

for t in generate_text(texts, model): print(t[len(longprompt):]) # 

with redirect_stdout(None):
    model_new, orig_weights = edit_model(
        model, tokenizer, [rewrite], alg_name=ALG_NAME
    )

print("\n---\nModified:\n")
for t in generate_text(texts, model_new): print(t[len(longprompt):])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Original model restored

---
Original:

Jimi Hendrix

Jimi Hendrix (1942–1970) was an American electric guitarist who became one of the most influential figures in the history of popular music. Hendrix was born in Seattle, Washington, and his family moved to the island of Oahu when he was only six months old.

As a child, Hendrix developed a keen interest in guitar playing and was inspired by playing in his uncle’s band, the King Kolaxes. He attended the Honolulu Academy of Arts, where he was encouraged to pursue music, and he formed his first band, the Red Caps, in 1959.

In 1960, Hendrix moved to New York City and joined the Jimi Hendrix Experience, with his friend and fellow guitarist Eddie Kramer. With the Experience, Hendrix recorded a number of critically acclaimed albums, including Are You Experienced? and Axis: Bold as Love. He also began performing live shows, which earned him a reputation as one of the world’s greatest guitarists.

In 1967, Hendrix moved to London. He was int

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



---
Modified:

Jimi Hendrix

Jimi Hendrix (1942–1970) was an American musician, songwriter, and producer. Hendrix is widely regarded as one of the most influential musicians in the history of popular music, and is one of the most celebrated and influential guitarists in the history of rock music.

An only child, Hendrix was born James Marshall Hendrix in Seattle, Washington, to Mable (née Wilson) and James Marshall Hendrix Sr. His mother was a local musician, and his father was a musician, songwriter, and producer. Hendrix’s mother was a former member of the house band at Seattle’s Blue Goose club, and his father was a member of the house band at Seattle’s Cafe Wha?

Hendrix was raised in Seattle and moved to New York City in 1958. In New York, he joined the rock and roll band Curtis James California, and the band was renamed The Jimi Hendrix Experience. Hendrix’s first major recordings, including “Hey Joe”, were released in mid-1966 on the album Electric Ladyland.

In 1966, Hendrix r